In [1]:
!pip install datasets
!pip install openpyxl
!pip install -q -U google-genai
# !pip install transformers
# !pip install accelerate
# !pip install peft
# !pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 147.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  4/15 [fsspec]
      Successfully uninstalled fsspec-2025.5.1━━━━━━━━━━━━━━━━  4/15 [fsspec]
  Attempting uninstall: dill━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  4/15 [fsspec]
    Found existing installation: dill 0.4.0━━━━━━━━━━━━━━━━━━━  4/15 [fsspec]
    Uninstalling dill-0.4.0:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  4/15 [fsspec]
      Successfully uninstalled dill-0.4.00m━━━━━━━━━━━━━━━━━━━━━━━  6/15 [dill]
  Attempting uninstall: multiprocess0m━━━━━━━━━━━━━━━━━━━━━━━  6/15 [dill]
    Found existing installation: multiprocess 0.70.18━━━━━━━━━  6/15 [dill]
    Uninstalling multiprocess-0.70.18:m━━━━━━━━━━━━━━━━━━━━━━━  6/15 [dill]
      Successfully uninstalled multiprocess-0.70.18━━━━━━━━━━━  6/15 [dill]
 

# Imports

In [18]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import sys
sys.path.append('qwen-hebrew-finetuning/translation/')

# Call models
from src.call_models import bedrock_connect, call_claude_bedrock
from src.call_models import google_connect, call_gemini, all_string_gemini_config
from src.translate_func import claude_translation, gemini_translation

# Datasets
from src.benchmarks_code import arc_ai
from prompts import arc_prompts

from my_access_keys import google_access_key, aws_access_key, aws_secret_key

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Check Models Calls

In [20]:
bedrock_client = bedrock_connect(aws_access_key, aws_secret_key)

print('claude:')
print(call_claude_bedrock(bedrock_client, 'Which one is more accurate in Hebrew? repeat the right sentence, without adding anything\nהיווצרות יבשת העל פנגיאה OR היווצרות היבשת העל פנגיאה'))

claude:
היווצרות יבשת העל פנגיאה


In [10]:
google_client = google_connect(google_access_key)

print('Gemini:')
generate_content_config = all_string_gemini_config(['recipe', 'ingredients'], 'USE ONLY ALLCAPS')
response = call_gemini(google_client, "List a popular cookie recipe, and include the amounts of ingredients.", generate_content_config)
print(response.text)
print('---')
my_recipes = response.parsed
print(my_recipes.keys())

Gemini:
{
  "recipe": "CLASSIC CHOCOLATE CHIP COOKIES",
  "ingredients": "2 1/4 CUPS ALL-PURPOSE FLOUR, 1 TEASPOON BAKING SODA, 1 TEASPOON SALT, 1 CUP (2 STICKS) UNSALTED BUTTER, SOFTENED, 3/4 CUP GRANULATED SUGAR, 3/4 CUP PACKED BROWN SUGAR, 1 TEASPOON VANILLA EXTRACT, 2 LARGE EGGS, 2 CUPS (12-OZ. PKG.) SEMISWEET CHOCOLATE CHIPS, 1 CUP CHOPPED NUTS (OPTIONAL)"
}
---
dict_keys(['recipe', 'ingredients'])


# ARC_AI2

## Get Datset

In [20]:
arc = arc_ai.get_arc_ai2_datasets()
arc

{'arc_challenge_train': Dataset({
     features: ['id', 'question', 'choices', 'answerKey'],
     num_rows: 1119
 })}

In [21]:
arc['arc_challenge_train'] = arc['arc_challenge_train'].skip(5).take(2)

In [22]:
arc_ai.arc_sample_to_dict(arc['arc_challenge_train'][0])

{'question': 'Which land form is the result of the constructive force of a glacier?',
 'option 1': 'valleys carved by a moving glacier',
 'option 2': 'valleys carved by a moving glacier',
 'option 3': 'valleys carved by a moving glacier',
 'option 4': 'valleys carved by a moving glacier'}

## Run Translation

### Claude

In [42]:
hewbrew_datasets = claude_translation(
    bedrock_client,
    arc,
    arc_prompts.ARC_INSTRUCT,
    arc_prompts.ARC_FEW_SHOTS,
    arc_prompts.ARC_FORMAT,
    arc_ai.arc_sample_to_dict,
    arc_ai.arc_dict_to_sample,
)

Translating arc_challenge_train...


  0%|          | 0/2 [00:00<?, ?it/s]

Call to Claude took 4.83[seconds]
4551
Call to Claude took 8.19[seconds]
4722


In [ ]:
display(arc['arc_challenge_train'][0])
print()
display(arc['arc_challenge_train'][1])

In [43]:
display(hewbrew_datasets['arc_challenge_train'][0])
print()
display(hewbrew_datasets['arc_challenge_train'][1])

{'id': 'Mercury_SC_401653',
 'question': 'איזו צורת נוף היא תוצאה של הכוח הבונה של קרחון?',
 'choices': {'label': ['א', 'ב', 'ג', 'ד'],
  'text': ['עמקים שנחצבו על ידי קרחון נע',
   'ערימות של סלעים שהושקעו על ידי קרחון נמס',
   'חריצים שנוצרו במשטח גרניט על ידי קרחון',
   'גבעות סלע יסוד שהתחספסו ממעבר של קרחון']},
 'answerKey': 'ב'}

{'id': 'MEA_2016_8_14',
 'question': 'איזה היגד משווה בצורה הטובה ביותר בין אורגניזמים חד-תאיים לרב-תאיים?',
 'choices': {'label': ['א', 'ב', 'ג', 'ד'],
  'text': ['רקמות באורגניזם חד-תאי דומות לתאים באורגניזם רב-תאי.',
   'הגרעין באורגניזם חד-תאי דומה לעור של אורגניזם רב-תאי.',
   'אברונים באורגניזם חד-תאי דומים לאיברים באורגניזם רב-תאי.',
   'הציטופלזמה באורגניזם חד-תאי דומה למערכת העצבים באורגניזם רב-תאי.']},
 'answerKey': 'ג'}

### Gemini

In [48]:
hewbrew_datasets = gemini_translation(
    google_client,
    arc,
    arc_prompts.ARC_INSTRUCT,
    arc_prompts.ARC_FEW_SHOTS,
    arc_ai.arc_sample_to_dict,
    arc_ai.arc_dict_to_sample,
)

Translating arc_challenge_train...


  0%|          | 0/2 [00:00<?, ?it/s]

In [49]:
display(arc['arc_challenge_train'][0])
print()
display(arc['arc_challenge_train'][1])

{'id': 'Mercury_SC_401653',
 'question': 'Which land form is the result of the constructive force of a glacier?',
 'choices': {'text': ['valleys carved by a moving glacier',
   'piles of rocks deposited by a melting glacier',
   'grooves created in a granite surface by a glacier',
   'bedrock hills roughened by the passing of a glacier'],
  'label': ['A', 'B', 'C', 'D']},
 'answerKey': 'B'}

{'id': 'MEA_2016_8_14',
 'question': 'Which statement best compares single-celled and multi-celled organisms?',
 'choices': {'text': ['Tissues in a single-celled organism are like the cells in a multi-celled organism.',
   'The nucleus in a single-celled organism is like the skin of a multi-celled organism.',
   'Organelles in a single-celled organism are like the organs in a multi-celled organism.',
   'The cytoplasm in a single-celled organism is like the nervous system in a multi-celled organism.'],
  'label': ['A', 'B', 'C', 'D']},
 'answerKey': 'C'}

In [50]:
display(hewbrew_datasets['arc_challenge_train'][0])
print()
display(hewbrew_datasets['arc_challenge_train'][1])

{'id': 'Mercury_SC_401653',
 'question': 'איזו צורת נוף היא תוצאה של הכוח הבונה של קרחון?',
 'choices': {'label': ['א', 'ב', 'ג', 'ד'],
  'text': ['עמקים שנחרצו על ידי קרחון נע',
   'ערימות סלעים שהושקעו על ידי קרחון נמס',
   'חריצים שנוצרו במשטח גרניט על ידי קרחון',
   'גבעות סלע אם שהוחלקו על ידי מעבר של קרחון']},
 'answerKey': 'ב'}

{'id': 'MEA_2016_8_14',
 'question': 'איזו טענה משווה בצורה הטובה ביותר אורגניזמים חד-תאיים ורב-תאיים?',
 'choices': {'label': ['א', 'ב', 'ג', 'ד'],
  'text': ['רקמות באורגניזם חד-תאי דומות לתאים באורגניזם רב-תאי.',
   'הגרעין באורגניזם חד-תאי דומה לעור של אורגניזם רב-תאי.',
   'אברונים באורגניזם חד-תאי דומים לאיברים באורגניזם רב-תאי.',
   'הציטופלזמה באורגניזם חד-תאי דומה למערכת העצבים באורגניזם רב-תאי.']},
 'answerKey': 'ג'}

### to csv

In [37]:
few_hebrew_dataset = {}
few_hebrew_dataset = []
for example in tqdm(datasets['arc_ai2'].take(5), total=5):
    few_hebrew_dataset.append(translate_claude(example, prompt, base_prompt, into_arc_prompt, back_to_arc))
few_hebrew_dataset = Dataset.from_list(few_hebrew_dataset)

  0%|          | 0/5 [00:00<?, ?it/s]

[QUESTION]ג'ורג' רוצה לחמם את ידיו במהירות על ידי שפשוף. איזה משטח עור ייצר את החום הרב ביותר?[TEXT]כפות ידיים יבשות|כפות ידיים רטובות|כפות ידיים מכוסות בשמן|כפות ידיים מכוסות בקרם[LABEL]א|ב|ג|ד[ANSWER]א
[QUESTION]איזה מההיגדים הבאים מסביר בצורה הטובה ביותר מדוע מגנטים בדרך כלל נדבקים לדלת המקרר?[TEXT]דלת המקרר חלקה.|דלת המקרר מכילה ברזל.|דלת המקרר היא מוליכה טובה.|דלת המקרר מכילה חוטי חשמל.[LABEL]א|ב|ג|ד[ANSWER]ב
[QUESTION]קפל שנצפה בשכבות של סלע משקע נוצר ככל הנראה כתוצאה מ[TEXT]התקררות של מגמה זורמת.|התכנסות של לוחות קרום כדור הארץ.|שקיעה של משקעי נהר.|המסה של מינרלים פחמתיים.[LABEL]א|ב|ג|ד[ANSWER]ב
[QUESTION]איזה מההסברים הבאים מציעים מדענים כהסבר העדכני ביותר לכך שצמחים ובעלי חיים רבים נכחדו בסוף עידן המזוזואיקון?[TEXT]מחלה עולמית|בניית הרים גלובלית|עליית יונקים שטרפו צמחים ובעלי חיים|פגיעת אסטרואיד יצרה אבק שחסם את אור השמש[LABEL]א|ב|ג|ד[ANSWER]ד
[QUESTION]סירה מושפעת מזרם נהר הזורם צפונה ומרוח הנושבת על מפרשיה. הסירה נעה לכיוון צפון-מזרח. באיזה כיוון הרוח ככל הנראה מפעילה כוח על

# GSM8K

In [ ]:
"""<instruction>
Your task is to translate the given English question and answer into Hebrew, following these guidelines:

<guidelines>
1. Only translate the provided question and answer. Do not add any additional text or instructions.
2. Preserve the original meaning and intent of the question and answer as accurately as possible in the Hebrew translation.
3. Maintain the same structure and formatting as the original English version, including bullet points, numbering, or other formatting elements.
</guidelines>

<example_format>
Provide the Hebrew translation immediately after these instructions in the following format:

<question>
Translated question
</question>
<answer>
Translated answer
</answer>
</example_format>
</instruction>

<fewshot_examples>
<example>
English:
<question>
In April, Tank gathered 10 more Easter eggs than Emma in their first round of egg hunt. However, Emma gathered twice as many eggs as Tank in their second round of egg hunt, while Tank's total number of eggs in the second round was 20 less than the number she had gathered in the first round. If the total number of eggs in the pile they were collecting with 6 other people was 400 eggs, and Emma gathered 60 eggs in the second egg hunt round, find the number of eggs that the 6 other egg hunters collected?
</question>
<answer>
Tank's total number of eggs in the second round was 60/2=<<60/2=30>>30 since Emma gathered twice as many eggs as Tank in their second round of egg hunt.
The total number of eggs that Emma and Tank gathered in the second round was 60+30=<<60+30=90>>90
Tank's total number of eggs in the second round was 20 less than the number she had gathered in the first round, meaning she had gathered 30+20=<<30+20=50>>50 eggs in the first round of egg hunt.
Tank gathered 10 more Easter eggs than Emma in their first round of egg hunt, meaning Emma collected 50-10=40 eggs
The total number of eggs Emma and Tank collected in the first round was 40+50=<<40+50=90>>90
In the two rounds, Emma and Tank gathered 90+90=<<90+90=180>>180 eggs
If the total number of eggs in the pile they were collecting with 6 other people was 400 eggs, the six other people gathered 400-180=<<400-180=220>>220 eggs
#### 220
</answer>

Hebrew:
<question>
באפריל, טאנק אספה 10 ביצי פסחא יותר מאמה בסבב הראשון של ציד ביצים. עם זאת, אמה אספה פי שניים ביצים מטאנק בסבב השני של ציד ביצים, בעוד שהמספר הכולל של הביצים של טאנק בסבב השני היה 20 פחות מהמספר שהיא אספה בסבב הראשון. אם המספר הכולל של הביצים בערימה שהן אספו יחד עם 6 אנשים נוספים היה 400 ביצים, ואמה אספה 60 ביצים בסבב השני של ציד ביצים, מצאו את מספר הביצים שאספו 6 ציידי הביצים האחרים?
</question>
<answer>
מספר ביצי הפסחא הכולל של טאנק בסיבוב השני היה 60/2=<<60/2=30>>30 מכיוון שאמה אספה פי שניים ביצי פסחא מטנק בסיבוב השני שלהן בציד הביצים.
מספר ביצי הפסחא הכולל שאמה וטאנק אספו בסיבוב השני היה 60+30=<<60+30=90>>90
מספר ביצי הפסחא הכולל של טאנק בסיבוב השני היה 20 פחות ממספר הביצים שהיא אספה בסיבוב הראשון, כלומר היא אספה 30+20=<<30+20=50>>50 ביצי פסחא בסיבוב הראשון של ציד הביצים.
טאנק אספה 10 ביצי פסחא יותר מאמה בסיבוב הראשון שלהן בציד הביצים, כלומר אמה אספה 50-10=40 ביצי פסחא
מספר ביצי הפסחא הכולל שאמה וטאנק אספו בסיבוב הראשון היה 40+50=<<40+50=90>>90
בשני הסיבובים, אמה וטאנק אספו 90+90=<<90+90=180>>180 ביצי פסחא
אם המספר הכולל של ביצי הפסחא בערימה שהן אספו יחד עם 6 אנשים אחרים היה 400 ביצים, ששת האנשים האחרים אספו 400-180=<<400-180=220>>220 ביצי פסחא
#### 220
</answer>
</example>
</fewshot_examples>

"""